# Get Nordstrom locations

#### Load Python tools and Jupyter config

In [1]:
import us
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from bs4 import BeautifulSoup
from vega_datasets import data as vega_data
from tqdm.notebook import tqdm, trange

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
place = "nordstrom"
place_formal = "Nordstrom"
color = "#000000"
today = pd.Timestamp.today().strftime("%Y-%m-%d")

---

## Scrape

#### Get a dataframe of ZIP Codes

In [4]:
zips_all = pd.read_json(
    "../../_reference/data/zips_reference_pop_gen.json"
).sort_values("population", ascending=False)

#### Get the most populous ZIPs and a sample of the rest

In [5]:
zips_top = zips_all.head(1000)
zips_sample = zips_all.tail(len(zips_all) - 1000).sample(1000)
zips_df = (
    pd.concat([zips_sample, zips_top])
    .reset_index(drop=True)
    .sort_values("population", ascending=False)
)
zips = zips_df["zip"].astype(str).str.zfill(5).to_list()

#### Headers for request

In [6]:
headers = {
    "authority": "api.nordstrom.com",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
}

In [7]:
extracted_info = []

for z in tqdm(zips):

    params = {
        "distance": "50",
        "sortOrder": "asc",
        "apikey": "Gneq2B6KqSbEABkg9IDRxuxAef9BqusJ",
    }

    try:
        response = requests.get(
            f"https://api.nordstrom.com/v2/store/postalcode/{z}",
            params=params,
            headers=headers,
        )

        locations = response.json()["stores"]

        for location in locations:
            store_number = location["number"]
            store_type = location["type"]
            store_name = location["name"]
            street = location["address"]
            city = location["city"]
            state = location["state"]
            zip_code = location["zipCode"]
            phone = location["phone"]
            latitude = location["latitude"]
            longitude = location["longitude"]
            url = location["rackStorePath"]
            timezone = location["timeZone"]

            location_info = {
                "store_number": store_number,
                "store_type": store_type,
                "store_name": store_name,
                "street": street,
                "city": city,
                "state": state,
                "zip": zip_code,
                "phone": phone,
                "latitude": latitude,
                "longitude": longitude,
                "url": url,
                "timezone": timezone,
            }
            extracted_info.append(location_info)

    except requests.exceptions.HTTPError as err:
        print(f"HTTP Error for ZIP code {z}: {err}")
    except Exception as e:
        print(f"HTTP Error for ZIP code {z}: {e}")

  0%|          | 0/2000 [00:00<?, ?it/s]

In [8]:
df = pd.DataFrame(extracted_info).drop_duplicates(subset="store_number")

#### How many stores? 

In [9]:
len(df)

358

#### The result

In [10]:
df.head()

,store_number,store_type,store_name,street,city,state,zip,phone,latitude,longitude,url,timezone
0,744,Rack,Nordstrom Rack Market at Town Center,2665 Town Center Boulevard North,Sugar Land,TX,77479,281.566.2000,29.593107,-95.619464,https://stores.nordstromrack.com/us/tx/sugar-land/2665-town-center-blvd-n,America/Chicago
1,748,Rack,Nordstrom Rack Bunker Hill,9714 Katy Freeway,Houston,TX,77055,281.605.2705,29.787470,-95.533556,https://stores.nordstromrack.com/us/tx/houston/9714-katy-freeway,America/Chicago
2,730,FullLineStore,Nordstrom Houston Galleria,5192 Hidalgo Street,Houston,TX,77056,832.201.2700,29.735854,-95.464215,,America/Chicago
3,741,Rack,Nordstrom Rack Centre at Post Oak,"5000 Westheimer Road, Suite 500",Houston,TX,77056,281.661.3600,29.742130,-95.463177,https://stores.nordstromrack.com/us/tx/houston/5000-westheimer-rd,America/Chicago
4,747,Rack,Nordstrom Rack Willowbrook Mall,8000 Willowbrook Mall,Houston,TX,77070,832.688.2000,29.974724,-95.570652,https://stores.nordstromrack.com/us/tx/houston/8000-willowbrook-mall,America/Chicago


#### Create a mapping of state abbreviations to full state names using the us library

In [11]:
state_mapping = {state.abbr: state.name for state in us.states.STATES}

#### New column of full state names based on abbreviations

In [12]:
df["state_name"] = df["state"].map(state_mapping)

#### Make sure our brand name gets in the dataframe

In [13]:
df["brand"] = place_formal

#### Add fetch date

In [ ]:
df["updated"] = today

#### Rack vs. full store vs. ?

In [27]:
df.store_type.value_counts().reset_index()

,store_type,count
0,Rack,257
1,FullLineStore,93
2,NordstromLocal,6
3,LastChance,2


---

## Geography

#### Make it a geodataframe

In [15]:
df_geo = df.copy()

In [16]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
).set_crs("4326")

---

## Maps

#### US states background

In [17]:
background = (
    alt.Chart(alt.topo_feature(vega_data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place_formal} locations")
    .project("albersUsa")
)

#### Location points map

In [18]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=5, color=color)
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [19]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value(color),
        tooltip=["state:N", "count:Q"],
    )
    .properties(
        title=f"Number of {place_formal} in US, by average lon/lat of locations"
    )
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### JSON

In [20]:
df.to_json(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.json",
    indent=4,
    orient="records",
)

#### CSV

In [21]:
df.to_csv(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.csv", index=False
)

#### GeoJSON

In [22]:
gdf.to_file(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.geojson",
    driver="GeoJSON",
)